In [3]:
def is_valid(state):
    return len(state) == 9 and sorted(state) == list(range(9))


def print_state(state):
    for i in range(0, 9, 3):
        print(' '.join(str(x) if x != 0 else ' ' for x in state[i:i+3]))
    print("-" * 5)


def manhattan_distance(state, goal_state):
    """
    Compute sum of Manhattan distances for all tiles from current to goal positions.
    """
    distance = 0
    for num in range(1, 9):  # ignore blank tile 0
        current_index = state.index(num)
        goal_index = goal_state.index(num)
        current_row, current_col = divmod(current_index, 3)
        goal_row, goal_col = divmod(goal_index, 3)
        distance += abs(current_row - goal_row) + abs(current_col - goal_col)
    return distance


def get_neighbors(state):
    neighbors = []
    zero_index = state.index(0)
    row, col = divmod(zero_index, 3)

    moves = {
        'Right': (0, 1),
        'Left': (0, -1),
        'Down': (1, 0),
        'Up': (-1, 0)
    }

    for move, (dr, dc) in moves.items():
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_zero_index = new_row * 3 + new_col
            new_state = list(state)
            new_state[zero_index], new_state[new_zero_index] = new_state[new_zero_index], new_state[zero_index]
            neighbors.append((tuple(new_state), move))
    return neighbors


def dfs_heuristic(initial_state, goal_state, max_depth=10000):
    """
    DFS with heuristic-guided neighbor exploration.
    Returns (path, moves) or (None, None).
    """
    stack = [(initial_state, [], [], 0)]  # (state, path, moves, depth)
    visited = set()

    while stack:
        current_state, path, moves, depth = stack.pop()

        if current_state == goal_state:
            return path + [current_state], moves

        if current_state not in visited and depth < max_depth:
            visited.add(current_state)
            neighbors = get_neighbors(current_state)

            # Sort neighbors by heuristic: prefer states closer to goal
            neighbors.sort(key=lambda x: manhattan_distance(x[0], goal_state), reverse=True)
            # reverse=True because stack is LIFO, we want the smallest heuristic neighbor to be popped first

            for neighbor_state, move in neighbors:
                stack.append((neighbor_state, path + [current_state], moves + [move], depth + 1))

    return None, None


def input_state(prompt):
    while True:
        raw = input(prompt)
        parts = raw.strip().split()
        if len(parts) != 9:
            print("Enter exactly 9 numbers separated by spaces.")
            continue
        try:
            numbers = tuple(int(x) for x in parts)
        except ValueError:
            print("Please enter integers only.")
            continue
        if not is_valid(numbers):
            print("Numbers must be 0 through 8 exactly once each.")
            continue
        return numbers


if __name__ == "__main__":
    print("Enter initial state (0 for blank):")
    initial_state = input_state("Enter 9 numbers separated by spaces: ")

    print("\nInitial state:")
    print_state(initial_state)

    print("Enter goal state (0 for blank):")
    goal_state = input_state("Enter 9 numbers separated by spaces: ")

    print("\nGoal state:")
    print_state(goal_state)

    solution_path, solution_moves = dfs_heuristic(initial_state, goal_state)

    if solution_path:
        print(f"\nSolution found in {len(solution_moves)} moves!\n")
        for i in range(len(solution_moves)):
            print(f"Move {i + 1}: {solution_moves[i]}")
            print_state(solution_path[i + 1])
    else:
        print("No solution found.")


Enter initial state (0 for blank):
Enter 9 numbers separated by spaces: 1 2 3 4 0 6 7 5 8

Initial state:
1 2 3
4   6
7 5 8
-----
Enter goal state (0 for blank):
Enter 9 numbers separated by spaces: 1 2 3 4 5 6 7 8 0

Goal state:
1 2 3
4 5 6
7 8  
-----

Solution found in 2 moves!

Move 1: Down
1 2 3
4 5 6
7   8
-----
Move 2: Right
1 2 3
4 5 6
7 8  
-----


In [10]:
def is_valid(state):
    return len(state) == 9 and sorted(state) == list(range(9))


def print_state(state):
    for i in range(0, 9, 3):
        print(' '.join(str(x) if x != 0 else ' ' for x in state[i:i+3]))
    print("-" * 5)


def manhattan_distance(state, goal_state):
    distance = 0
    for num in range(1, 9):
        current_index = state.index(num)
        goal_index = goal_state.index(num)
        current_row, current_col = divmod(current_index, 3)
        goal_row, goal_col = divmod(goal_index, 3)
        distance += abs(current_row - goal_row) + abs(current_col - goal_col)
    return distance


def get_neighbors(state):
    neighbors = []
    zero_index = state.index(0)
    row, col = divmod(zero_index, 3)

    moves = {
        'Right': (0, 1),
        'Left': (0, -1),
        'Down': (1, 0),
        'Up': (-1, 0)
    }

    for move, (dr, dc) in moves.items():
        new_row, new_col = row + dr, col + dc
        if 0 <= new_row < 3 and 0 <= new_col < 3:
            new_zero_index = new_row * 3 + new_col
            new_state = list(state)
            new_state[zero_index], new_state[new_zero_index] = new_state[new_zero_index], new_state[zero_index]
            neighbors.append((tuple(new_state), move))
    return neighbors


def depth_limited_dfs(current_state, goal_state, depth_limit, path, moves, visited, last_visited):
    """
    Recursive Depth-Limited DFS.
    Returns (path, moves) if goal found, else None.
    Updates last_visited with last state seen.
    """
    last_visited[0] = current_state  # Update last visited state

    if current_state == goal_state:
        return path + [current_state], moves

    if depth_limit == 0:
        return None

    visited.add(current_state)

    neighbors = get_neighbors(current_state)
    neighbors.sort(key=lambda x: manhattan_distance(x[0], goal_state))

    for neighbor_state, move in neighbors:
        if neighbor_state not in visited:
            result = depth_limited_dfs(neighbor_state, goal_state, depth_limit - 1,
                                       path + [current_state], moves + [move], visited, last_visited)
            if result:
                return result

    visited.remove(current_state)
    return None


def iterative_deepening_search(initial_state, goal_state, max_depth):
    """
    IDS: Increase depth from 0 up to max_depth.
    Returns (path, moves, last_state)
    """
    last_state = None

    for depth in range(max_depth + 1):
        print(f"Trying depth limit: {depth}")
        visited = set()
        last_visited = [None]  # Mutable container to hold last visited state
        result = depth_limited_dfs(initial_state, goal_state, depth, [], [], visited, last_visited)
        if result:
            path, moves = result
            return path, moves, None
        if depth == max_depth and last_visited[0] is not None:
            last_state = last_visited[0]

    return None, None, last_state


def input_state(prompt):
    while True:
        raw = input(prompt)
        parts = raw.strip().split()
        if len(parts) != 9:
            print("Enter exactly 9 numbers separated by spaces.")
            continue
        try:
            numbers = tuple(int(x) for x in parts)
        except ValueError:
            print("Please enter integers only.")
            continue
        if not is_valid(numbers):
            print("Numbers must be 0 through 8 exactly once each.")
            continue
        return numbers


if __name__ == "__main__":
    print("Enter initial state (0 for blank):")
    initial_state = input_state("Enter 9 numbers separated by spaces: ")

    print("\nInitial state:")
    print_state(initial_state)

    print("Enter goal state (0 for blank):")
    goal_state = input_state("Enter 9 numbers separated by spaces: ")

    while True:
        try:
            max_depth = int(input("\nEnter max depth for iterative deepening search: "))
            if max_depth < 0:
                print("Please enter a non-negative integer.")
                continue
            break
        except ValueError:
            print("Please enter a valid integer.")

    solution_path, solution_moves, last_state = iterative_deepening_search(initial_state, goal_state, max_depth)

    if solution_path:
        print(f"\nSolution found in {len(solution_moves)} moves!\n")
        for i in range(len(solution_moves)):
            print(f"Move {i + 1}: {solution_moves[i]}")
            print_state(solution_path[i + 1])
    else:
        print("\nNo solution found within the given depth limit.")
        if last_state:
            print("Last explored state at max depth:")
            print_state(last_state)


Enter initial state (0 for blank):
Enter 9 numbers separated by spaces: 1 2 3 7 4 6 5 8 0

Initial state:
1 2 3
7 4 6
5 8  
-----
Enter goal state (0 for blank):
Enter 9 numbers separated by spaces: 1 2 3 4 5 6 7 8 0

Enter max depth for iterative deepening search: 4
Trying depth limit: 0
Trying depth limit: 1
Trying depth limit: 2
Trying depth limit: 3
Trying depth limit: 4

No solution found within the given depth limit.
Last explored state at max depth:
1 4 2
7   3
5 8 6
-----
